# Tree Lab

## 准备工作
### 环境准备
请确保完成以下依赖包的安装，并且通过下面代码来导入与验证。

In [1]:
import pandas as pd
import numpy as np

### 数据集准备
我们将使用以下数据集进行决策树的构建。该数据集包括7个特征，以及一个标签“是否适合读博”，这些特征描述了适合读博的各种条件，如love doing research,I absolutely want to be a college professor等。

请运行下面的代码来加载数据集。

（防侵权说明）参考https://zhuanlan.zhihu.com/p/372884253，数据集来源GPT4，但构造的决策树不一定与参考内容完全一致。

In [2]:
# read decision_tree_datasets.csv
train_data = pd.read_csv('train_phd_data.csv')
test_data = pd.read_csv('test_phd_data.csv')


## 决策树构建 (10 分)
在这个部分，你将学习并完成决策树的构建。注意：不考虑剪枝，决策树构建停止条件是数据所有实例属于同一类或者特征不可再分（即每个特征值都一样）。

我们采用信息增益率作为分类标准，同时也允许使用其他指标，如基尼系数。

请完成以下函数：

1. **计算数据的信息熵** `getInfoEntropy()`
2. **根据选取的特征进行数据分割** `split_data()`
3. **根据分类标准找到最优特征** `find_best_feature()`

你可能会用到`pandas`库函数，请参考 [pandas官方文档](https://pandas.pydata.org/docs/reference/)。

In [27]:

def getInfoEntropy(data):
    ''' 
        calculate the information entropy of the data

    Args:
        data: the data set, the last column is the label, the other columns are the features

    Returns:
        Entropy: float, the information entropy of the data
    '''
    
    Entropy = 0.0
    
    # TODO: 1. count the number of different labels samples (each class has how many samples) -- count_class
    ## hint: use pd.value_counts() to count the number of different labels samples
    ## hint: use data.iloc[:,-1] to get the last column of the data
    count_class = data.iloc[:, -1].value_counts()
    # print(data.iloc[:, -1])
    
    # TODO: 2. calculate the number of data
    data_count = len(data)
    

    for count in count_class:
        # TODO: 3. calculate the probability of each class
        p = count / data_count
        
        # TODO: 4. calculate the entropy of each class
        Entropy -= p * np.log2(p)
    #print('当前数据集的信息熵为：',Entropy)
    return Entropy


In [28]:
## test getInfoEntropy
print(getInfoEntropy(train_data))

0.8256265261578954


In [29]:
def split_data(data, column):
    ''' 
        split the data set according to the feature column

        Args:
            data: the data set, the last column is the label, the other columns are the features
            column: the feature column
        Returns:
            splt_datas: Series, the data set after splitting
    '''
    # 1. construct a Series to save the data set after splitting
    splt_datas = pd.Series(dtype='object')  
    # 2. get the unique values of the feature column
    str_values = data.iloc[:,column].unique()  
    # 3. find the data set corresponding to each unique value
    for i in range(len(str_values)):   
        df = data.loc[data.iloc[:,column] == str_values[i]]

        splt_datas[str(i)] = df
    return splt_datas
    



In [30]:
def find_best_feature(data):

    '''  
        find the best feature to split the data set

        Args:
            data: the data set, the last column is the label, the other columns are the features
        Returns:
            best_feature: the best feature
            best_Series: Series, the data set after splitting
    '''
    best_feature_index = 0    
    baseEnt = getInfoEntropy(data)  
    bestInfoGain_ratio = 0.0
    numFeatures = data.shape[1] - 1
    # print(numFeatures)  
    InfoGain = 0.0 
    
    # Loop through each feature to calculate information gain ratio.
    for i in range(numFeatures):
        # if(i==6):
        #     print(f"----------Start in Feature {i}---------")
        newEnt = 0.0
        # avoid div 0 error
        IV = 1e-5
        # TODO: 1. split the data set according to the feature column
        series = split_data(data, i)
        # print(series)

        # 2. calculate the information entropy of each data set, and calculate the weighted average information entropy
        for j in range(len(series)):
            # if(i==6):
            #     print(f"----------Mid in i = {i}, j = {j}---------")
            df = series[j]
            
            # TODO: 3. calculate the probability of each data set
            p = len(df) / float(len(data))

            # if(i==6 and j==1):
            #     print(len(df.iloc[:, -1].value_counts()))
            #     print(len(df))
            #     print(p)
            #     Entropy = 0.0
            #     count_class = df.iloc[:, -1].value_counts()
            #     print(count_class)
            #     data_count = len(df)
            #     print(data_count)
                
            #     for count in count_class:
            #         print(count)
            #         # print(f"----------Mid, {k}---------")
            #         # TODO: 3. calculate the probability of each class
            #         p = count / data_count
            #         print(p)
            #         # TODO: 4. calculate the entropy of each class
            #         Entropy -= p * np.log2(p)

            #     print(Entropy)

            # TODO: 4. calculate the weighted average information entropy
            newEnt += p * getInfoEntropy(df)
    
            
            # TODO: 5. calculate the entropy of class labels IV
            IV -= p * np.log2(p) if p > 0 else 0
        
        # TODO: 6. calculate the information gain 
        InfoGain = baseEnt - newEnt
        
        # TODO: 7. calculate the information gain ratio
        InfoGain_ratio = InfoGain / IV
        # print(InfoGain_ratio)
        # if(i==6):
        print(u"第%d个特征的信息增益率为：%.3f" % (i, InfoGain_ratio))

        if InfoGain_ratio > bestInfoGain_ratio:
            bestInfoGain_ratio = InfoGain_ratio
            best_feature_index = i
            best_Series = series
        
        # if(i==6):
        #     print(f"----------Result of Finding Best Feature {i}---------")
        # print(data.columns[best_feature_index])
        # print(best_Series)
        # best_Series = {val: data[data.iloc[:, best_feature_index] == val] for val in data.iloc[:, best_feature_index].unique()}
        # print(best_Series)

    print("All Done")
    return data.columns[best_feature_index], best_Series


In [34]:
#create decision tree
def creat_Tree(data):

    '''
        create decision tree

        Args:
            data: the data set, the last column is the label, the other columns are the features
        Returns:
            Tree: dict, the decision tree
    '''
    # get the class labels of the data set
    y_values = data.iloc[:,-1].unique()   

    # TODO: 1. If there is only one class label, stop splitting and return the class label.
    if len(y_values) == 1:
        return y_values[0]
    
    # 2. Check if the value of each feature is the same. If so, return the class label with the most samples.
    flag = 0
    for i in range(data.shape[1]-1):   
        if(len(data.iloc[:,i].unique()) != 1):
            flag = 1
            break
    
    # TODO: 3. If all features are identical, return the class label with the most samples.
    if(flag == 0):
        # value_count = data.iloc[:, -1].mode()[0]
        value_count = data.iloc[:, -1].value_counts().idxmax()
        return value_count
        # return 0

    # 4. TODO: Find the best feature to split the data set.
    
    best_feature, best_Series = find_best_feature(data)
    print("----------Return of Finding Best Feature---------")
    print(best_feature)
    print(best_Series)

    Tree = {best_feature:{}}
    # 5. Build the tree recursively. 
    # for value, subset in best_Series.items():
    #     # split_data = best_Series.iloc[j]
        
    #     # # read the value of the best feature
    #     # value = split_data.loc[:,best_feature].unique()[0]  

    #     # # delete the best feature 
    #     # split_data = split_data.drop(best_feature, axis = 1) 
    #     subset = subset.drop(columns=best_feature, axis=1)
        
    #     # TODO: 6. recursively call the function to build the tree
    #     Tree[best_feature][value] = creat_Tree(subset) 
    for j in range(len(best_Series)):
        split_data = best_Series[j]
        value = split_data[best_feature].iloc[0]
        split_data = split_data.drop(best_feature, axis=1)
        Tree[best_feature][value] = creat_Tree(split_data)
        
    return Tree


In [35]:
Tree = creat_Tree(train_data)

第0个特征的信息增益率为：0.146
第1个特征的信息增益率为：0.226
第2个特征的信息增益率为：0.050
第3个特征的信息增益率为：0.003
第4个特征的信息增益率为：0.127
第5个特征的信息增益率为：0.003
第6个特征的信息增益率为：0.017
All Done
----------Return of Finding Best Feature---------
I absolutely want to be a college professor
0        I love doing research  I absolutely want t...
1        I love doing research  I absolutely want t...
dtype: object
第0个特征的信息增益率为：0.023
第1个特征的信息增益率为：0.046
第2个特征的信息增益率为：0.046
第3个特征的信息增益率为：0.079
第4个特征的信息增益率为：0.055
第5个特征的信息增益率为：0.079
All Done
----------Return of Finding Best Feature---------
I am OK being with judged all the time
0        I love doing research  Money is important ...
1        I love doing research  Money is important ...
dtype: object
第0个特征的信息增益率为：0.000
第1个特征的信息增益率为：0.088
第2个特征的信息增益率为：0.088
第3个特征的信息增益率为：0.130
第4个特征的信息增益率为：0.354
All Done
----------Return of Finding Best Feature---------
I work 9-5 Mon-Fri
0        I love doing research  Money is important ...
1        I love doing research  Money is important ...
dtype: object
第0个特征的信

In [36]:
# visualize decision tree
from graphviz import Digraph


def plot_tree(tree, parent_name, node_id=0, graph=None, edge_label=''):
    
    
    if graph is None:
        graph = Digraph(comment='Decision Tree')

    
    if not isinstance(tree, dict):
        current_node_name = f'node{node_id}' 
        graph.node(current_node_name, label=str(tree))
        graph.edge(parent_name, current_node_name, label=edge_label)
        node_id += 1
        return node_id
    
    for k, v in tree.items():
        current_node_name = f'node{node_id}' 
        node_label = f'{k}' if isinstance(v, (str, int)) else k
        graph.node(current_node_name, label=node_label)
        graph.edge(parent_name, current_node_name, label=str(edge_label))

        if isinstance(v, dict):
            for key in v:
                # 假设分支可以用 '0' 和 '1' 来区分
                node_id += 1
                node_id = plot_tree(v[key], current_node_name, node_id, graph, edge_label=str(key))
                
    return node_id

# plot decision tree
tree_graph = Digraph(comment='Decision Tree')
plot_tree(Tree, 'Root', 0, tree_graph)
tree_graph.render('decision_tree', format='png', cleanup=True)

'decision_tree.png'

In [37]:
# classfiy test data
def classify(tree, test_data):
    ''' 
        classify test data

        Args:
            tree: dict, the decision tree
            test_data: the test data set, the last column is the label, the other columns are the features
        Returns:
            class_label: the predicted class label
    '''
    ## get the checked feature of the decision tree
    first_str = list(tree.keys())[0]
    
    ## get the index of the feature
    feat_index = test_data.index.get_loc(first_str)

    # get the value of the feature to be checked in the decision tree
    key = test_data.iloc[feat_index]

    # TODO: get the subtree corresponding to the value of the feature
    subtree = tree[first_str].get(key)

    # TODO: recursively call the function to classify the test data
    # hint: if the value of the feature is not a dict, it means that the decision tree has reached the leaf node, and the value of the feature is the predicted class label
    if isinstance(subtree, dict):
        return classify(subtree, test_data)
    else:
        return subtree  # Predicted class label
    pass

def test(test_data, tree):
    right_label = []
    for i in range(len(test_data)):
        sample = test_data.iloc[i]
        
        # whether the predicted class label is equal to the true class label
        if classify(tree, sample) == sample[-1]:
            # if equal, the predicted is 1
            right_label.append(1)
        else:
            right_label.append(0)
    
    acc = sum(right_label) / len(right_label)
    print('accuracy: ', acc)
    

In [38]:
test(test_data, Tree)
test(train_data, Tree)

accuracy:  0.8571428571428571
accuracy:  1.0


接下来我们将进行一项小测试，目的在于评估您是否适合攻读博士学位。

请注意！这仅仅是基于假设的模型，无法准确预测实际情况。请将其视为一次轻松的尝试，仅供娱乐之用，不要用其来替代对自身状况的思考与决策。

In [39]:
# You can input your profile to predict your phd admission result
# input your profile about "I love doing research,I absolutely want to be a college professor,Money is important to me,I can deal with extreme stress and competition,I am OK being with judged all the time,I need a clear target and immediate feedback,I work 9-5 Mon-Fri"
loving = input('Do you love research? (1/0)')
professor = input('Do you want to be a professor? (1/0)')
money = input('Is money important to you? (1/0)')
stress = input('Can you deal with stress? (1/0)')
judge = input('Can you deal with being judged all the time? (1/0)')
feedback = input('Do you need a clear target and immediate feedback? (1/0)')
work = input('Do you work 9-5 Mon-Fri? (1/0)')

# Combine the user's responses into a single data frame
test_data = pd.Series({
    'I love doing research': int(loving),
    'I absolutely want to be a college professor': int(professor),
    'Money is important to me': int(money),
    'I can deal with extreme stress and competition': int(stress),
    'I am OK being with judged all the time': int(judge),
    'I need a clear target and immediate feedback': int(feedback),
    'I work 9-5 Mon-Fri': int(work)
})

# Use the decision tree to predict the result 
result = classify(Tree, test_data)

# Print the result to the user
if result == 1:
    print("Congratulations! According to the model, you are likely to gain admission for Ph.D.")
elif result == 0: 
    print("Unfortunately, according to the model, you are unlikely to gain admission for Ph.D.")


Congratulations! According to the model, you are likely to gain admission for Ph.D.
